## T2

In [ ]:
import sys, re
sys.path.append('../../code/python')
import numpy as np
import pandas as pd
import helpers as hh
import matplotlib.pyplot as plt
from matplotlib.figure import SubplotParams
from matplotlib import cm, colors
from IPython.core.debugger import Tracer
%matplotlib inline

D = pd.HDFStore('../../data/tables/station_data.h5')
T = hh.stationize(D['ta_c'].xs('prom', 1, 'aggr').drop(10, 1, 'elev')) + 273.15
sta = D['sta']
S = pd.HDFStore('../../data/tables/LinearLinear.h5')
Tm = S['T2']
B = Tm.add( -T ).mean(1)
Z = S['z'][['d03_op' if re.search('d03_\d', z) else z for z in B.columns]]
Z.columns = B.columns
dz = (sta['elev'] - Z.T).T
Map = hh.basemap()

In [ ]:
def plot(B, clims):
    for k, b in B.iteritems():
        for j, x in enumerate(B.minor_axis):
            ax = plt.subplot(3, B.shape[2], k * B.shape[2] + j + 1)
            D = pd.concat((sta.loc[B.major_axis, ('lon', 'lat')], b[x]), axis=1).dropna()
            if k == 0:
                ax.set_title(x)
            plt.set_cmap('coolwarm')
            Map.scatter(D['lon'].as_matrix(), D['lat'].as_matrix(), c=D.iloc[:,-1], marker='o', latlon=True)
            Map.drawcoastlines()
            Map.drawparallels(range(-32, -28, 1), labels=[max(B.shape[2] - 2 - j, 0), 0, 0, 0])
            Map.drawmeridians(range(-72, -69, 1), labels=[0, 0, 0, max(k - 1, 0)])
            plt.clim(-clims[k], clims[k])
            if j == B.shape[2] - 1:
                bb = ax.get_position()
                plt.colorbar(
                    cax=fig.add_axes([bb.x1 + 0.02, bb.y0, 0.02, bb.y1 - bb.y0]))

In [ ]:
fig = plt.figure(
    figsize=(12, 8),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))

plot(pd.Panel({0:dz, 1:B, 2:B - 0.0065 * dz}), clims=[1750,13,13])

In [ ]:
fig = plt.figure(
    figsize=(8, 8),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))

x = ['d02', 'd03_orl', 'd03_0_00', 'd03_0_12']
plot(pd.Panel({0:dz[x], 1:B[x], 2:B[x] - 0.0065 * dz[x]}), clims=[1150, 10, 10])

In [ ]:
x = ['fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(10, 8),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))

day = lambda x: x.groupby(lambda y: y.date).min()

dt = day(T) # daily min of station data
P = pd.Panel(
    {
        0: day(Tm[x]).add(-dt).mean(1), # daily min of model data minus daily min of station data
        1: day( Tm[x].add( -0.0065 * dz[x], 1) ).add(-dt).mean(1), # model data corrected by standard lapse rate
        2: day( Tm[x].add(-B[x], 1) ).add(-dt).mean(1) # model data with mean bias removed
    }
)

plot(P, clims=[10, 10, 10])

In [ ]:
x = ['fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(10, 8),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))

day = lambda x: x.groupby(lambda y: y.date).max()

dt = day(T) # daily min of station data
P = pd.Panel(
    {
        0: day(Tm[x]).add(-dt).mean(1), # daily min of model data minus daily min of station data
        1: day( Tm[x].add( -0.0065 * dz[x], 1) ).add(-dt).mean(1), # model data corrected by standard lapse rate
        2: day( Tm[x].add(-B[x], 1) ).add(-dt).mean(1) # model data with mean bias removed
    }
)

plot(P, clims=[10, 10, 10])